In [1]:
from imports_func import *
from dbase import *

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X=np.load('/home/miquel/ForMiquel/CLEAN2/X_c1_1j_full2.npy') 
y=np.load('/home/miquel/ForMiquel/CLEAN2/y_1j_full2.npy') #subject labels
sub_id=np.load('/home/miquel/ForMiquel/CLEAN2/sub_id_short.npy') #subject idxs
jac_idx=np.load('/home/miquel/ForMiquel/CLEAN2/jac_idx_1j_full2.npy')

In [3]:
# build vector with dt's
dt=[]
for i in range(0,len(sub_id)):
    dt.append(Subjects[sub_id[i]].jacobians[jac_idx[i]].dt)

dt = np.asarray(dt)/365 
#dt=np.reshape(dt, (124,1))

In [7]:
def metrics_repetitions_folds(X, y, folds, splits, dt, percentage):
    
    l2=X.shape[0]
    X=np.concatenate((X,np.reshape(y, (l2,1))), axis=1)
    SPECIFICITY=[]
    PRECISION=[]
    RECALL=[]
    AUC=[]
    F=[]
    tp_l=[]
    fp_l=[]
    fn_l=[]
    tn_l=[]
    
    for i in range(0,splits):
        
        print("repetition:", i)
        X_aux = np.take(X,np.random.permutation(X.shape[0]),axis=0,out=X)
        y_aux = X_aux[:,-1]
        X_aux= X_aux[:,:-1]
        print(np.shape(X_aux))
        
        dt=np.reshape(dt, (l2,1))
        kf = cross_validation.KFold(l2, n_folds=folds, shuffle=True, random_state=i)    
        
        fold = 0
        
        for train_index, test_index in kf:  
            X_train = X_aux[train_index,:]
            X_test = X_aux[test_index,:]
            y_train = y_aux[train_index]
            y_test = y_aux[test_index]
            dt_test = dt[test_index]
            print(test_index)
            print(y_test)

            print("fold:", fold)
            fold = fold + 1

            S = SelectPercentile(percentile=percentage)
            S.fit(X_train, y_train)
        
            X_train_s = X_train[:,S.get_support()]
            X_test_s = X_test[:,S.get_support()]
            
            clf = LogisticRegression(solver='liblinear', penalty='l2', C=10e-6, class_weight={0: 0.42, 1:0.58})
            clf.fit(X_train_s, y_train)
            pred = clf.predict(X_test_s)
            probas_ = clf.predict_proba(X_test_s)
            
            mask0 = np.where(dt_test>1.09)#~400 days
            mask0 = np.asarray(mask0)
            pred = pred[mask0][0]
            y_test = y_test[mask0][0]
            probas_ = probas_[:, 1]
            probas_ = probas_[mask0][0]
              
            conf = confusion_matrix(y_test, pred)
            print(conf)
            fpr, tpr, thresholds = roc_curve(y_test, probas_)
            roc_auc = sklearn.metrics.auc(fpr, tpr)
            AUC.append(roc_auc)
            
            tp_l.append(conf[1,1])
            fp_l.append(conf[0,1])
            fn_l.append(conf[1,0])
            tn_l.append(conf[0,0])
    
    
    TP = np.sum(tp_l)
    FP = np.sum(fp_l)
    TN = np.sum(tn_l)
    FN = np.sum(fn_l)
    
    F=(2*TP)/(2*TP + FP + FN)
    RECALL=TP / (TP + FN)
    SPECIFICITY= TN / (TN + FP)
    PRECISION=TP / (TP + FP)
    
    #f1=f1_score(y_test, pred)
    #F.append(f1)
    #precision=precision_score(y_test, pred)
    #recall=recall_score(y_test, pred)
    #PRECISION.append(precision)
    #RECALL.append(recall)
    #specificity= (conf[0,0])/(conf[0,0]+conf[0,1])
    #SPECIFICITY.append(specificity)
    
    print("F_mean:", F)
    print("AUC_mean:", np.mean(AUC))
    print("PRECISION_mean:", PRECISION)
    print("RECALL_mean:", RECALL)
    print("SPECIFICITY_mean:", SPECIFICITY)
    
    #print("F_mean:", np.mean(F))
    #print("AUC_mean:", np.mean(AUC))
    #print("PRECISION_mean:", np.mean(PRECISION))
    #print("RECALL_mean:", np.mean(RECALL))
    #print("SPECIFICITY_mean:", np.mean(SPECIFICITY))
    
    #print("F_std:", np.std(F))
    print("AUC_std:", np.std(AUC))
    #print("PRECISION_std:", np.std(PRECISION))
    #print("RECALL_std:", np.std(RECALL))
    #print("SPECIFICITY_std:", np.std(SPECIFICITY))
            
            
    return F, AUC, PRECISION, RECALL, SPECIFICITY        


In [9]:
folds = 5
splits = 20
percentage = 1.75
F, AUC, PRECISION, RECALL, SPECIFICITY = metrics_repetitions_folds(X, y, folds, splits, dt, percentage)

repetition: 0
(124, 572738)
[  2   7   8  10  16  22  24  43  45  48  50  54  56  61  63  71  84  86
  90  91  93  97  98  99 111]
[ 1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.
  1.  1.  0.  0.  0.  0.  1.]
fold: 0


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [     4     13     90 ..., 572207 572510 572528] are constant.
  UserWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


[[4 4]
 [3 4]]
[  3   6  11  13  18  26  27  30  33  51  59  60  62  66  68  75  78  94
 105 108 114 115 118 121 123]
[ 1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.
  1.  1.  0.  0.  1.  1.  0.]
fold: 1


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     99    290 ..., 572728 572729 572730] are constant.
  UserWarning)


[[5 0]
 [4 3]]
[  0   1   4   5  15  17  23  28  34  35  38  40  41  42  52  53  55  73
  76  89  92  96 101 107 113]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.  0.  1.  0.  1.  0.
  1.  0.  0.  1.  1.  0.  1.]
fold: 2


...

[[5 0]
 [4 3]]
repetition: 19
(124, 572738)
[  2   8  13  16  18  27  30  36  37  47  64  65  67  77  79  82  83  91
  96 101 104 106 112 116 121]
[ 1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.
  0.  1.  0.  1.  0.  0.  1.]
fold: 0


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     99    204 ..., 572728 572729 572730] are constant.
  UserWarning)


[[ 1 10]
 [ 1  4]]
[  1   6  14  20  28  29  33  38  43  46  58  59  61  70  72  76  78 102
 109 111 113 118 119 120 122]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  1.  0.
  0.  0.  0.  0.  1.  0.  1.]
fold: 1
[[0 9]
 [1 3]]
[  0   3   7  11  21  23  24  26  41  42  44  48  51  62  63  87  88  89
  90  94  95  99 100 108 115]
[ 1.  0.  0.  1.  0.  1.  1.  0.  1.  1.  0.  0.  0.  1.  0.  1.  1.  0.
  0.  0.  1.  0.  0.  0.  0.]
fold: 2


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     99    149 ..., 572407 572421 572704] are constant.
  UserWarning)


[[7 3]
 [2 2]]
[  4   9  12  17  25  32  35  45  49  53  54  55  57  60  68  69  73  75
  81  84  85  92 107 114 123]
[ 1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.  0.
  1.  1.  0.  0.  0.  1.  0.]
fold: 3
[[9 0]
 [7 1]]
[  5  10  15  19  22  31  34  39  40  50  52  56  66  71  74  80  86  93
  97  98 103 105 110 117]
[ 0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  1.  0.  1.  1.  1.  1.  0.  1.
  1.  1.  0.  1.  0.  0.]
fold: 4
[[6 1]
 [6 2]]
F_mean: 0.452857142857
AUC_mean: nan
PRECISION_mean: 0.393788819876
RECALL_mean: 0.532773109244
SPECIFICITY_mean: 0.460773480663
AUC_std: nan
